In [1]:
import pandas as pd
import numpy as np
from  sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

train = pd.read_csv('E:\\Dev\\AV\\train_votes.csv')
train = train.drop(train[train.Views > 3000000].index)


In [3]:
labelencoder_X = LabelEncoder()
train['Tag'] = labelencoder_X.fit_transform(train['Tag'])
train.drop(['ID','Username'], axis=1,inplace =True)
target = train['Upvotes']

In [4]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([train['Answers']])[0]
train['pd_watched'] = pd_watched

In [5]:
feature_names = [x for x in train.columns if x not in ['Upvotes']]
x_train, x_test, y_train, y_test = train_test_split(train[feature_names], target,test_size = 0.22,random_state =205)
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

In [6]:
st = sc_X.transform(x_test)

In [10]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(x_train, y_train, verbose=False)
# make predictions
predictions = my_model.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error( y_test,predictions)))
print("R2 Score : " + str(r2_score(y_test, predictions)))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Mean Absolute Error : 168.01354142729278
R2 Score : 0.8534901497156416


In [15]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)

my_model.fit(x_train, y_train, early_stopping_rounds=20, 
             eval_set=[(x_test, y_test)], verbose=False)
from sklearn.metrics import mean_absolute_error
    
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_test)))
print("R2 Score : " + str(r2_score(y_test, predictions)))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Mean Absolute Error : 168.01354142729278
R2 Score : 0.8534901497156416


In [12]:
print(r2_score(y_test, predictions))
# testing
test = pd.read_csv('E:\\Dev\\AV\\test_votes.csv')
ids = test['ID']
test.drop(['ID','Username'], axis=1,inplace =True)
#25-March - 0.903437879332785
#next 0.9034534322501852
#0.9037949838052866 - lb .794 

0.8534901497156416


In [13]:
labelencoder_X = LabelEncoder()
test['Tag'] = labelencoder_X.fit_transform(test['Tag'])
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([test['Answers']])[0]
test['pd_watched'] = pd_watched

In [16]:
test = sc_X.fit_transform(test)
pred_test = my_model.predict(test)
pred_test=abs(pred_test)

In [18]:
import datetime

submission = pd.DataFrame({'ID': ids,
                           'Upvotes':pred_test
                           })

submission.to_csv("submit_" +str( datetime.date.today())+ "_nr.csv",index=False)

In [ ]:
import os
cwd = os.getcwd()
cwd

In [ ]:
0.9034534322501852 - 0.903437879332785 

In [ ]:
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model.coordinate_descent import Lasso, \
    LassoCV, ElasticNet, ElasticNetCV, MultiTaskLasso, MultiTaskElasticNet, \
    MultiTaskElasticNetCV, MultiTaskLassoCV, lasso_path, enet_path
from sklearn.linear_model.coordinate_descent import Lasso, \
    LassoCV
from sklearn.linear_model import LassoLarsCV, lars_path

from sklearn.model_selection import train_test_split
#xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)
#X_train, X_test, y_train, y_test = train_test_split(train[feature_names], target, test_size = 0.22,random_state =205)
def test_lasso_cv():
    #X_train, y_train, X_test, y_test = build_dataset()
    X_train, y_train, X_test, y_test = train_test_split(train[feature_names], target, test_size = 0.22,random_state =205)
    clf = LassoCV(n_alphas=10, eps=1e-3, max_iter=150, precompute=True)
    clf.fit(X_train, y_train)
    
    # Check that the lars and the coordinate descent implementation
    # select a similar alpha
    lars = LassoLarsCV(normalize=False, max_iter=30).fit(X_train, y_train)

    #mse_lars = interpolate.interp1d(lars.cv_alphas_, lars.mse_path_.T)
    # test set
    print(clf.score(X_test, y_test))
    print(lars.score(X_test, y_test))

In [ ]:
def build_dataset(n_samples=50, n_features=200, n_informative_features=10,
                  n_targets=1):
    """
    build an ill-posed linear regression problem with many noisy features and
    comparatively few samples
    """
    random_state = np.random.RandomState(0)
    if n_targets > 1:
        w = random_state.randn(n_features, n_targets)
    else:
        w = random_state.randn(n_features)
    w[n_informative_features:] = 0.0
    X = random_state.randn(n_samples, n_features)
    y = np.dot(X, w)
    X_test = random_state.randn(n_samples, n_features)
    y_test = np.dot(X_test, w)
    return X, y, X_test, y_test
 

In [ ]:
test_lasso_cv()